# This notebook is for data preparation and data mining

Delivered data: train_data_seq_atc_codes - train_data_seq_atc_codes.csv

In [1]:
import pandas as pd
import numpy as np

## Open the delivered csv. This is drug buying data.

In [2]:
data = pd.read_csv('train_data_seq_atc_codes - train_data_seq_atc_codes.csv',sep=';')

In [3]:
data.head()

,X0,X1,X2,X3,X4,X5,X6,X7,X8,X9
0,19,44,24,12,0,0,0,0,0,0
1,37,12,1,0,0,0,0,0,0,0
2,3,40,13,0,0,0,0,0,0,0
3,3,4,27,0,0,0,0,0,0,0
4,3,13,1,0,0,0,0,0,0,0


In [4]:
data = data.drop_duplicates()

In [5]:
#This is dictionary to 'decode' numbers to ATC codes
decode = {"1": "C07AB",
          "2": "C10AA",
          "3": "C09AA",
          "4": "A02BC",
          "5": "B01AC",
          "6": "C03CA",
          "7": "C08CA",
          "8": "A10BA",
          "9": "N02BE",
          "10": "A12BA",
          "11": "C09CA",
          "12": "M01AE",
          "13": "C03BA",
          "14": "C03DA",
          "15": "N06BX",
          "16": "M04AA",
          "17": "M02AA",
          "18": "R05X",
          "19": "N02BA",
          "20": "A10BB",
          "21": "R05CB",
          "22": "C09DA",
          "23": "N02AX",
          "24": "N07CA",
          "25": "R02AA",
          "26": "G04CA",
          "27": "A05BA",
          "28": "N05BB",
          "29": "M01AB",
          "30": "C07AG",
          "31": "H03AA",
          "32": "A03AD",
          "33": "A10AD",
          "34": "M03BX",
          "35": "B01AA",
          "36": "N06AB",
          "37": "A02BA",
          "38": "A01AD",
          "39": "A03AX",
          "40": "C10AB",
          "41": "D08AX",
          "42": "C05CA",
          "43": "N05CF",
          "44": "R01AA",
          "45": "A03AA",
          "46": "C09BB",
          "47": "A07DA",
          "48": "N06AX",
          "49": "A05AX",
          "50": "M01AC"}

In [6]:
#Now I am mining for real names for ATC codes with upper classes

from bs4 import BeautifulSoup as soup
from urllib.request import urlopen

atc_descriptions = dict()

def description_parser(code):
    try:
        site = 'https://www.whocc.no/atc_ddd_index/?code='+code
        my_url = site
        sauce = urlopen(my_url).read()
        potato = soup(sauce, 'lxml')
        if code not in atc_descriptions:
            if len(code)==1:
                atc_descriptions[code] = potato.b.text.lower()
            else:
                atc_descriptions[code] = str(potato.find(href="./?code="+code).text).lower()
    except:
        return None

for x in decode:
    description_parser(decode.get(x))
    description_parser(decode.get(x)[0])
    description_parser(decode.get(x)[:3])
    description_parser(decode.get(x)[:4])

In [7]:
atc_descriptions

{'C': 'cardiovascular system',
 'A': 'alimentary tract and metabolism',
 'B': 'blood and blood forming organs',
 'N': 'nervous system',
 'M': 'musculo-skeletal system',
 'R': 'respiratory system',
 'G': 'genito urinary system and sex hormones',
 'H': 'systemic hormonal preparations, excl. sex hormones and insulins',
 'D': 'dermatologicals'}

In [8]:
#save to csv
pd.DataFrame(atc_descriptions.values(),index=atc_descriptions.keys()).to_csv('atc_descriptions.csv')

In [9]:
#here I am mining for names of all substances codes and names belongs to every delivered ATC codes
import re

substances_list = dict()

for i in decode:
    group_code = decode.get(i)
    site = 'https://www.whocc.no/atc_ddd_index/?code='+group_code
    my_url = site
    sauce = urlopen(my_url).read()
    potato = soup(sauce, 'lxml')
    for i in potato.find_all(href=re.compile(group_code))[2:]:
        code_x = str(i)[17:24]
        code_y = i.text
        if code_x not in substances_list:
            substances_list[code_x] = code_y
substances_list

{'C07AB01': 'practolol',
 'C07AB02': 'metoprolol',
 'C07AB03': 'atenolol',
 'C07AB04': 'acebutolol',
 'C07AB05': 'betaxolol',
 'C07AB06': 'bevantolol',
 'C07AB07': 'bisoprolol',
 'C07AB08': 'celiprolol',
 'C07AB09': 'esmolol',
 'C07AB10': 'epanolol',
 'C07AB11': 's-atenolol',
 'C07AB12': 'nebivolol',
 'C07AB13': 'talinolol',
 'C07AB14': 'landiolol',
 'C10AA01': 'simvastatin',
 'C10AA02': 'lovastatin',
 'C10AA03': 'pravastatin',
 'C10AA04': 'fluvastatin',
 'C10AA05': 'atorvastatin',
 'C10AA06': 'cerivastatin',
 'C10AA07': 'rosuvastatin',
 'C10AA08': 'pitavastatin',
 'C09AA01': 'captopril',
 'C09AA02': 'enalapril',
 'C09AA03': 'lisinopril',
 'C09AA04': 'perindopril',
 'C09AA05': 'ramipril',
 'C09AA06': 'quinapril',
 'C09AA07': 'benazepril',
 'C09AA08': 'cilazapril',
 'C09AA09': 'fosinopril',
 'C09AA10': 'trandolapril',
 'C09AA11': 'spirapril',
 'C09AA12': 'delapril',
 'C09AA13': 'moexipril',
 'C09AA14': 'temocapril',
 'C09AA15': 'zofenopril',
 'C09AA16': 'imidapril',
 'A02BC01': 'omepraz

In [10]:
#save to csv
pd.DataFrame(substances_list.values(),index=substances_list.keys()).to_csv('substances_list.csv')

In [11]:
substance_vendor_data = pd.DataFrame(columns=['Code','Name','Vendor'])

In [12]:
#here I am mining for all substances vendors- just for case
import re
import json
import requests


for substance_code in substances_list:
    substance_name = substances_list.get(substance_code)
    substance_name_replaced = substance_name.replace(' ','_',substance_name.count(' '))
    try:
        site = 'https://pubchem.ncbi.nlm.nih.gov/compound/'+substance_name_replaced
        my_url = site
        sauce = urlopen(my_url).read()
        potato = soup(sauce, 'lxml')
        cid_code = potato.head.find('meta',attrs={'name':"pubchem_uid_value"}).attrs['content']
        site = 'https://pubchem.ncbi.nlm.nih.gov/rest/pug_view/categories/compound/'+str(cid_code)+'/JSON/?response_type=display'
        my_url = requests.get(site)
        sauce = json.loads(my_url.text)
        for x in sauce["SourceCategories"]['Categories'][0]['Sources']:
            new_row = {'Code':substance_code, 'Name':substance_name, 'Vendor':x.get('SourceName')}
            substance_vendor_data = substance_vendor_data.append(new_row, ignore_index=True)
    except:
        pass
substance_vendor_data.drop_duplicates(inplace=True)

In [13]:
substance_vendor_data.head()

,Code,Name,Vendor
0,C07AB01,practolol,Angel Pharmatech Ltd.
1,C07AB01,practolol,BLD Pharm
2,C07AB01,practolol,LGC Standards
3,C07AB01,practolol,"AHH Chemical co.,ltd"
4,C07AB01,practolol,3B Scientific (Wuhan) Corp


In [14]:
#save to csv
substance_vendor_data.to_csv('substance_vendor_data.csv')

In [15]:
data.head()

,X0,X1,X2,X3,X4,X5,X6,X7,X8,X9
0,19,44,24,12,0,0,0,0,0,0
1,37,12,1,0,0,0,0,0,0,0
2,3,40,13,0,0,0,0,0,0,0
3,3,4,27,0,0,0,0,0,0,0
4,3,13,1,0,0,0,0,0,0,0


In [16]:
#adding all values from delivered csv file

data_length = len(data)
binary_med_data = pd.DataFrame(columns=decode.values())

for x in range(data_length):
    row_med = dict()
    n = 0
    for i in data.iloc[x].values:
        if decode.get(str(i)) == None or decode.get(i) == 0:
            break
        else:
            row_med[decode.get(str(i))] = 1
            n += 1
    row_med['bought_qty'] = n
    binary_med_data = binary_med_data.append(row_med,ignore_index=True)
binary_med_data = binary_med_data.fillna(0)
binary_med_data = binary_med_data.astype('int32')

In [17]:
binary_med_data.head()

,C07AB,C10AA,C09AA,A02BC,B01AC,C03CA,C08CA,A10BA,N02BE,A12BA,...,C05CA,N05CF,R01AA,A03AA,C09BB,A07DA,N06AX,A05AX,M01AC,bought_qty
0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,4
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
3,0,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
4,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3


In [18]:
binary_med_data.shape

(68981, 51)

In [19]:
data.shape

(68981, 10)

In [20]:
#saving to csv
binary_med_data.to_csv('binary_med_data.csv')

In [21]:
#now I am making datasets by upper classes

In [22]:
cols_level1 = []
cols_level2 = []
cols_level3 = []
for x in binary_med_data.columns[:-1]:
    if x[0] not in cols_level1:
        cols_level1.append(x[0])
    if x[:3] not in cols_level2:
        cols_level2.append(x[:3])
    if x[:4] not in cols_level3:
        cols_level3.append(x[:4])
print(cols_level1, len(cols_level1))
print(cols_level2, len(cols_level2))
print(cols_level3, len(cols_level3))

['C', 'A', 'B', 'N', 'M', 'R', 'G', 'H', 'D'] 9
['C07', 'C10', 'C09', 'A02', 'B01', 'C03', 'C08', 'A10', 'N02', 'A12', 'M01', 'N06', 'M04', 'M02', 'R05', 'N07', 'R02', 'G04', 'A05', 'N05', 'H03', 'A03', 'M03', 'A01', 'D08', 'C05', 'R01', 'A07'] 28
['C07A', 'C10A', 'C09A', 'A02B', 'B01A', 'C03C', 'C08C', 'A10B', 'N02B', 'A12B', 'C09C', 'M01A', 'C03B', 'C03D', 'N06B', 'M04A', 'M02A', 'R05X', 'R05C', 'C09D', 'N02A', 'N07C', 'R02A', 'G04C', 'A05B', 'N05B', 'H03A', 'A03A', 'A10A', 'M03B', 'N06A', 'A01A', 'D08A', 'C05C', 'N05C', 'R01A', 'C09B', 'A07D', 'A05A'] 39


In [23]:
bought_level1 = pd.DataFrame(columns=cols_level1)
bought_level2 = pd.DataFrame(columns=cols_level2)
bought_level3 = pd.DataFrame(columns=cols_level3)

#Note I will sum repeated higher classes because It can tell how many types of meds someone bought. 

In [ ]:
for i in range(len(binary_med_data)):
    i_dict = dict()
    ii_dict = dict()
    iii_dict = dict()
    for y in cols_level1:
        i_dict[y] = 0
        for x in binary_med_data.columns:
            if y == x[:len(y)] and binary_med_data[x].iloc[i] == 1:
                i_dict[y] = i_dict.get(y) + 1
    for y in cols_level2:
        ii_dict[y] = 0
        for x in binary_med_data.columns:
            if y == x[:len(y)] and binary_med_data[x].iloc[i] == 1:
                ii_dict[y] = ii_dict.get(y) + 1
    for y in cols_level3:
        iii_dict[y] = 0
        for x in binary_med_data.columns:
            if y == x[:len(y)] and binary_med_data[x].iloc[i] == 1:
                iii_dict[y] = iii_dict.get(y) + 1
    bought_level1 = bought_level1.append(i_dict,ignore_index=True)
    bought_level2 = bought_level2.append(ii_dict,ignore_index=True)
    bought_level3 = bought_level3.append(iii_dict,ignore_index=True)
bought_level1 = bought_level1.astype('int32')
bought_level2 = bought_level2.astype('int32')
bought_level3 = bought_level3.astype('int32')

In [ ]:
bought_level1.to_csv('bought_level1.csv')
bought_level1.head()

In [ ]:
bought_level2.to_csv('bought_level2.csv')
bought_level2.head()

In [ ]:
bought_level3.to_csv('bought_level3.csv')
bought_level3.head()

# Summary:

Note about ATC code: https://en.wikipedia.org/wiki/Anatomical_Therapeutic_Chemical_Classification_System

### 1. The delivered data contains data for every medicine that has been bought by every customer. These medicines had an ATC code of chemical/therapeutic/pharmacological subgroup (level 4 in ATC code).

### 2. There was a lack of information about meaning of these codes, so they has been parsed. Additionaly information about every vendor of substance belonging to the code has been also gained.

### 3. Delivered data has been transformed for more clear data (every column is ATC code):
        - binary_med_data (level 4 ATC code- data directly from original data)
        - bought_level1 (level 1 ATC code- made from binary_med_data)
        - bought_level2 (level 2 ATC code- made from binary_med_data)
        - bought_level3 (level 3 ATC code- made from binary_med_data)

#### The next chapter: ATC- data visualisations